# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(geo_1.info())
geo_1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
display(geo_2.info())
geo_2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
display(geo_3.info())
geo_3.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## Обучение и проверка модели

In [6]:
#Функция, разделяет выборку на признаки и цели , отсеевает не нужные данные, делит на тренировочную и валидационную выборки,
#Обучает модель и выводит показатели rmse и средний запас, возвращает датафрэйм с раельными и предсказанными данными
def split_fit (data):
    features = data.drop(['product','id'], axis=1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_valid)
    
    print('rmse = ',(mean_squared_error(predictions, target_valid))**(0.5))
    print('Средняя запас = ',predictions.mean())
    
    real_predictions = pd.DataFrame()
    real_predictions['real'] = target_valid
    real_predictions['predictions'] = predictions
    
    return real_predictions

In [7]:
geo_pred_1 = split_fit(geo_1)
geo_pred_1.head(5)

rmse =  37.5794217150813
Средняя запас =  92.59256778438035


,real,predictions
71751,10.038645,95.894952
80493,114.551489,77.572583
2655,132.603635,77.892640
53233,169.072125,90.175134
91141,122.325180,70.510088


In [8]:
geo_pred_2 = split_fit(geo_2)
geo_pred_2.head(5)

rmse =  0.893099286775617
Средняя запас =  68.728546895446


,real,predictions
71751,80.859783,82.663314
80493,53.906522,54.431786
2655,30.132364,29.748760
53233,53.906522,53.552133
91141,0.000000,1.243856


In [9]:
geo_pred_3 = split_fit(geo_3)
geo_pred_3.head(5)

rmse =  40.02970873393434
Средняя запас =  94.96504596800489


,real,predictions
71751,61.212375,93.599633
80493,41.850118,75.105159
2655,57.776581,90.066809
53233,100.053761,105.162375
91141,109.897122,115.303310


Предсказания оказались довольно близки к реальным данным, самый  высокий результат у второго региона RMSE 0.89 это очень подозрительный результат. Есть вероятность что есть прямая зависимость между какой то из характеристик и объемом запасов, судя по тому что зависмости в других регионах нету можно предположить что данный второго региона не корректны.

## Подготовка к расчёту прибыли

In [10]:
BUDGET = 10**10 
PRICE_BARREL = 450 * 10**3
N = 200
TOTAL_N = 500

In [11]:
total_required = BUDGET / PRICE_BARREL
avg_top = total_required / N

print('Всего требудется:', total_required,'тыс. баралей ')
print('Стреднее двухсот лучших должен быть не менее:', avg_top,'тыс. баралей')

Всего требудется: 22222.222222222223 тыс. баралей 
Стреднее двухсот лучших должен быть не менее: 111.11111111111111 тыс. баралей


Средний запас в каждом из регионов меньше необходимого.

## Расчёт прибыли и рисков 

In [12]:
#Функция для подщёта прибыли
def revenue(volumes, price=PRICE_BARREL, min_volume=avg_top): 
    
    revenue_per_well =  (volumes - min_volume) * price
    
    revenue_total = np.sum(revenue_per_well)
    return revenue_total

In [13]:
# находит и выводит доверительные интервалы, среднюю прибыли и риск убытка 
def risk (geo_pred):
    risk_s = []
    random = np.random.RandomState(54321)
    for i in range(1000):
        predictions_sample =(geo_pred.sample(500, replace=True, random_state=random)
                             .sort_values(by='predictions', ascending=False)
                             .head(200))
        risk_s.append(revenue(predictions_sample['real']))
        
    risk_s = pd.Series(risk_s)   
    lower = risk_s.quantile(0.025)
    upper = risk_s.quantile(0.975)
    
    print('Доверительный интервал', lower, ' ', upper)
    print('Средняя прибыли:',risk_s.mean(),'руб.')
    print(f'Рис убытков: {(risk_s[risk_s < 0].count()/len(risk_s)*100)}%')
    
    return risk_s

In [14]:
risk_geo_1 = risk(geo_pred_1)

Доверительный интервал -110864002.8152088   903359833.6663692
Средняя прибыли: 392082908.8547081 руб.
Рис убытков: 6.4%


In [15]:
risk_geo_2 = risk(geo_pred_2)

Доверительный интервал 33826729.38230054   846567069.2162611
Средняя прибыли: 437334151.9785654 руб.
Рис убытков: 1.5%


In [16]:
risk_geo_3 = risk(geo_pred_3)

Доверительный интервал -174131176.37829503   893656461.3281599
Средняя прибыли: 378510561.31141555 руб.
Рис убытков: 9.0%


Для разработки лучше всего подходит второй регион , так как у него наименьший риск и наибольшая средняя прибыль.